In [ ]:
# Importamos las librerías necesarias para trabajar

import pandas as pd # Librería Pandas para trabajar los DataFrame
from googleapiclient.discovery import build # API de Analytics
from oauth2client.service_account import ServiceAccountCredentials # Acceso con credenciales de Google Cloud
from datetime import date, timedelta # Para el calculo de fechas
import numpy as np # Libreria de matematicas
from decimal import Decimal # Correcion de datos que vienen desde Analytics para ser leidos como enteros en Python
import os
import time
from functools import reduce
import warnings
warnings.filterwarnings("ignore")

In [34]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly'] # El URL de la API de analytics
KEY_FILE_LOCATION = 'proyecto-prueba-367719-06da1516dd39.json' # El JSON con la información proporcionada de Google Cloud
# El ID se encuentra en la configuración por página de Analytics, es un ID por página

def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

# Funcion que recolecta la informacion 
def get_report(analytics, pageTokenVar, metricas, dimensiones, fecha_inicio, fecha_final, VIEW_ID):
    try:
        a = analytics.reports().batchGet(
          body={
            'reportRequests': [
            {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': fecha_inicio + 'daysAgo', 'endDate': fecha_final + 'daysAgo'}],
              'metrics': metricas,
              'dimensions': dimensiones,
              'pageSize': 100000000,
              'pageToken': pageTokenVar,
              'samplingLevel': 'LARGE'
            }]
          }
      ).execute()
    except (HttpError, timeout):
        time.sleep(35)
        a = analytics.reports().batchGet(
          body={
            'reportRequests': [
            {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': fecha_inicio + 'daysAgo', 'endDate': fecha_final + 'daysAgo'}],
              'metrics': metricas,
              'dimensions': dimensiones,
              'pageSize': 100000000,
              'pageToken': pageTokenVar,
              'samplingLevel': 'LARGE'
            }]
          }
        ).execute()
    return a

# Funcion que recolecta la informacion     
def handle_report(analytics, pagetoken, rows, metricas, dimensiones, fecha_inicio, fecha_final, VIEW_ID):  
    response = get_report(analytics, pagetoken, metricas, dimensiones, fecha_inicio, fecha_final, VIEW_ID)

    # Encabezados (dimensiones y metricas)
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    # Paginacion
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    # Filas
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew

    # Consulta recursiva de la siguiente pagina
    if pagetoken != None:
        return handle_report(analytics, pagetoken, rows, metricas, dimensiones, fecha_inicio, fecha_final, VIEW_ID)
    else:
        nicerows=[]
        for row in rows:
            dic={}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dic[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dic[metric.get('name')] = float(Decimal(value))
                    else:
                        dic[metric.get('name')] = int(Decimal(value))
            nicerows.append(dic)
        return nicerows
    
# Funcion para calcular el numero de dias entre ayer y la primera fecha de Analytics
def fechas(n):
    today = date.today() - timedelta(1)
    Fecha_creacion_google = date(2005, 11, 14)
    diferencia = today - Fecha_creacion_google
    dias = diferencia.days
    fechas_separadas = np.arange(0, dias, n).tolist()
    if fechas_separadas[-1] != dias:\
        fechas_separadas.append(dias)
    fechas_separadas = fechas_separadas[::-1]
    fechas_separadas[-1] = 1
    return fechas_separadas

# Funcion obsoleta
def reportes_por_fecha(fecha_registrada, fecha_registrada_1, VIEW_ID, metricas, dimensiones):
    i = 0
    df_inicio = []
    while i < len(dimensiones):
        rows = []
        rows = handle_report(analytics,'0',rows, metricas, dimensiones[i], str(fecha_registrada), str(fecha_registrada_1), VIEW_ID)
        df = pd.DataFrame(list(rows)).set_index('ga:clientID')
        df_inicio.append(df)
        if i > 0:
            df_inicio[i] = df_inicio[i].drop('ga:sessionCount', axis=1)
            df_inicio[i] = df_inicio[i].drop('ga:sessions', axis=1)
            df_inicio[i] = df_inicio[i].drop('ga:sessionDuration', axis=1)
            df_inicio[0] = df_inicio[0].merge(df_inicio[i], left_index=True, right_index=True, how='outer')
            df_inicio[0] = df_inicio[0].drop_duplicates()
        i += 1
    df_por_fecha = df_inicio[0]
    del(df_inicio)
    return df_por_fecha

# Funcion obsoleta
def reportes_por_fecha_1(fecha_registrada, fecha_registrada_1, VIEW_ID, metricas, dimensiones):
    i = 0
    df_lista=[]
    while i < len(dimensiones):
        rows = []
        rows = handle_report(analytics,'0',rows, metricas, dimensiones[i], str(fecha_registrada), str(fecha_registrada_1), VIEW_ID)
        df = pd.DataFrame(list(rows)).set_index('ga:clientID')
        if i > 0:
            df = df.drop(columns=['ga:sessionCount', 'ga:sessions', 'ga:sessionDuration'])
        df_lista.append(df)
        i += 1
    return df_lista
        

def reportes_por_fecha_pagina(fecha_registrada, fecha_registrada_1, VIEW_ID, metricas, dimensiones):
    i = 0
    df_inicio = []
    while i < len(metricas):
        rows = []
        rows = handle_report(analytics,'0',rows, metricas[i], dimensiones, str(fecha_registrada), str(fecha_registrada_1), VIEW_ID)
        df = pd.DataFrame(list(rows))
        df_inicio.append(df)
        i += 1
    return df_inicio

def metricas_dimensiones(valor):
    if valor == 0:
        metricas_usuario = [
                    {'expression': 'ga:sessions'}, {'expression': 'ga:sessionDuration'}
                   ]
        dimensiones_usuario = [
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'},
                         {'name': 'ga:userType'},    
                         {'name': 'ga:daysSinceLastSession'}],
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'}, 
                         {'name': 'ga:browser'},  
                         {'name': 'ga:browserVersion'},
                         {'name': 'ga:operatingSystem'},
                         {'name': 'ga:operatingSystemVersion'}],
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'},
                         {'name': 'ga:deviceCategory'},
                         {'name': 'ga:browserSize'}],
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'},
                         {'name': 'ga:mobileDeviceBranding'},
                         {'name': 'ga:mobileDeviceModel'},
                         {'name': 'ga:mobileInputSelector'},
                         {'name': 'ga:mobileDeviceInfo'}],
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'},
                         {'name': 'ga:subContinent'},
                         {'name': 'ga:country'},
                         {'name': 'ga:region'},
                         {'name': 'ga:city'},
                         {'name': 'ga:latitude'},
                         {'name': 'ga:longitude'}],
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'},
                         {'name': 'ga:javaEnabled'},
                         {'name': 'ga:language'},
                         {'name': 'ga:screenResolution'}],
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'},
                         {'name': 'ga:pagePath'},
                         {'name': 'ga:pageTitle'},
                         {'name': 'ga:secondPagePath'},
                         {'name': 'ga:pageDepth'}],
                        [{'name': 'ga:clientID'},
                         {'name': 'ga:sessionCount'},
                         {'name': 'ga:date'},
                         {'name': 'ga:hour'},
                         {'name': 'ga:minute'}]
                        ]
        return metricas_usuario, dimensiones_usuario
    elif valor == 1:
        metricas_pagina = [
                    [{'expression': 'ga:users'}, 
                     {'expression': 'ga:newUsers'},
                     {'expression': 'ga:avgSessionDuration'}, 
                     {'expression': 'ga:sessionsPerUser'},
                     {'expression': 'ga:hits'}, 
                     {'expression': 'ga:entrances'}],
                    [{'expression': 'ga:pageviews'}, 
                     {'expression': 'ga:uniquePageviews'},
                     {'expression': 'ga:avgSessionDuration'}, 
                     {'expression': 'ga:avgPageLoadTime'},
                     {'expression': 'ga:avgPageDownloadTime'}, 
                     {'expression': 'ga:avgServerResponseTime'}]
                   ]
        dimensiones_pagina = [{'name': 'ga:pagePath'}, {'name': 'ga:pageTitle'}]
        return metricas_pagina, dimensiones_pagina

def reportes_usuario(z):
    metricas, dimensiones = metricas_dimensiones(0)
    id = pd.read_csv('id.csv')
    VIEW_ID = str(id.iloc[z][3])
    titulo = str(id.iloc[z][0]) + '-' + str(id.iloc[z][1]) + '-' + str(id.iloc[z][2])
    print('Archivo:', titulo, '(reporte por usuario)')
    j = 0
    fecha = fechas(100)
    indice = 0
    while j < len(fecha) - 1:
        try:
            df_lista = reportes_por_fecha_1(fecha[j], fecha[j + 1], VIEW_ID, metricas, dimensiones)
            print(fecha[j], 'y', fecha[j + 1])
            i = 0
            while i < len(df_lista):
                df_lista[i].to_csv('produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID + '/df_' + str(j) + '_' + str(i) + '.csv')
                i += 1
            print('Ok')
            indice = len(df_lista)
            del(df_lista)
            j += 1
        except KeyError:
            j += 1
        except HttpError:
            time.sleep(35)
            j -= 1
        except timeout:
            j -= 1
        except ValueError:
            print("La página no tiene ninguna vista registrada dentro de estas fechas en Analytics")
    return indice
        
def concatenar(z, indice):
    print('Concatenando archivos...')
    id = pd.read_csv('id.csv')
    VIEW_ID = str(id.iloc[z][3])
    titulo = str(id.iloc[z][0]) + '-' + str(id.iloc[z][1]) + '-' + str(id.iloc[z][2])
    i = 0
    while i < indice:
        lista = []
        nombres = []
        for file in os.listdir(os.path.abspath(os.getcwd() + '/produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID)):
            if file.endswith(str(i) + ".csv"):
                nombres.append(file)
                lista.append(pd.read_csv(os.getcwd() + '/produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID + '/' + file))    
        df_concatenado = pd.concat(lista)
        df_concatenado.to_csv('produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID + '/df_' + str(i) + '_final.csv', encoding='utf8')
        for nombre in nombres:
            os.remove('produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID + '/' + nombre)
        del(df_concatenado)
        del(lista)
        del(nombres)
        i += 1
    nombres = []
    for file in os.listdir(os.path.abspath(os.getcwd() + '/produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID)):
        if file.endswith(".csv"):
            nombres.append(file)
    if nombres == []:
        print("La página no tiene ninguna vista registrada dentro de Analytics")
        f = open('produccion/usuario/' + str(z) + '_' + titulo + '_' + VIEW_ID + '/error.txt','w')
        f.write('La página no tiene ninguna vista registrada dentro de Analytics')
        f.close()
    else:
        pass
    del(id)
    del(VIEW_ID)
    del(titulo)
    
def reportes_pagina(z):
    metricas, dimensiones = metricas_dimensiones(1)
    id = pd.read_csv('id.csv')
    VIEW_ID = str(id.iloc[z][3])
    titulo = str(id.iloc[z][0]) + '-' + str(id.iloc[z][1]) + '-' + str(id.iloc[z][2])
    print('Archivo:', titulo, '(reporte por página)')
    fecha = fechas(40)
    i = 0
    while i < len(fecha) - 1:
        rows = []
        rows = handle_report(analytics,'0',rows, metricas[0], dimensiones, str(fecha[i]), str(fecha[i + 1]), VIEW_ID)
        dfanalytics_1 = pd.DataFrame(list(rows))
        dfanalytics_1['Periodo_dias'] = str(fecha[i]) + ' a ' + str(fecha[i+1])
        dfanalytics_1.to_csv('cache/df_' + str(i) + '_1.csv')
        
        rows_2 = []
        rows_2 = handle_report(analytics,'0',rows_2, metricas[1], dimensiones, str(fecha[i]), str(fecha[i + 1]), VIEW_ID)
        dfanalytics_2 = pd.DataFrame(list(rows_2))
        dfanalytics_2['Periodo_dias'] = str(fecha[i]) + ' a ' + str(fecha[i+1]) 
        dfanalytics_2.to_csv('cache/df_' + str(i) + '_2.csv')
        
        if i == 0:
            print('0% ...')
        elif i == 75:
            print('50% ...')
        elif i == 154:
            print('100%')

        i += 1
        
    dfs = []
    dfs = pd.DataFrame(list(dfs))
    nombres = []
    print('concatenando... 0%')
    for file in os.listdir(os.path.abspath(os.getcwd() + '/cache')):
        if file.endswith("1.csv"):
            nombres.append(file)
    i = 0
    while i < len(nombres):
        df = pd.read_csv('cache/' + nombres[i])
        dfs = pd.concat([df, dfs])
        os.remove('cache/' + nombres[i]) 
        i += 1
    
    dfs = dfs.drop_duplicates().drop(columns=['Unnamed: 0'])
    dfs.to_csv('produccion/pagina/' + str(z) + '_' + titulo + '_1.csv', encoding='utf8')

    print('concatenando... 50%')
    dfs = []
    dfs = pd.DataFrame(list(dfs))
    nombres = []
    for file in os.listdir(os.path.abspath(os.getcwd() + '/cache')):
        if file.endswith("2.csv"):
            nombres.append(file)
    i = 0
    while i < len(nombres):
        df = pd.read_csv('cache/' + nombres[i])
        dfs = pd.concat([df, dfs])
        os.remove('cache/' + nombres[i]) 
        i += 1

    dfs = dfs.drop_duplicates().drop(columns=['Unnamed: 0'])
    dfs.to_csv('produccion/pagina/' + str(z) + '_' + titulo + '_2.csv', encoding='utf8')
    print('concatenando... 100%')
    del(dfs)
    del(nombres)
    
    for file in os.listdir(os.path.abspath(os.getcwd() + '/produccion/pagina')):
        if file.endswith('.csv'):
            peso = os.path.getsize(os.path.abspath(os.getcwd() + '/produccion/pagina/' + file))
            if peso < 50:
                os.remove(os.path.abspath(os.getcwd() + '/produccion/pagina/' + file))
                f = open('produccion/pagina/' + str(z) + '_' + titulo + '_error.txt', 'w')
                f.write('La página no tiene ninguna vista registrada dentro de Analytics')
                f.close()
            else:
                pass
        
def generador_carpetas():
    id = pd.read_csv('id.csv')
    try:
        os.mkdir('produccion')
    except FileExistsError:
        pass
    try:
        os.mkdir('produccion/usuario')
    except FileExistsError:
        pass
    try:
        os.mkdir('produccion/pagina')
    except FileExistsError:
        pass
    try:
        os.mkdir('cache')
    except FileExistsError:
        pass

    z = 0
    while z < len(id):
        VIEW_ID = str(id.iloc[z][3])
        titulo = str(id.iloc[z][0]) + '-' + str(id.iloc[z][1]) + '-' + str(id.iloc[z][2])
        try:
            os.mkdir('produccion/usuario/' + str(z) + '_' + titulo + '_' +  VIEW_ID)
        except FileExistsError:
            pass
        z += 1
    del(id)
    del(VIEW_ID)
    del(titulo)
        
analytics = initialize_analyticsreporting()
generador_carpetas()

In [ ]:
i = 0
df_id = pd.read_csv('id.csv')
sitios = len(df_id)
del(df_id)
while i < sitios:
    print(i)
    x = reportes_usuario(i)
    concatenar(i, x)
    reportes_pagina(i)
    i += 1

In [7]:
df = pd.read_csv('id.csv')
df

,Cuenta_de_Analytics,Propiedades_y_aplicaciones,Vistas,View_ID
0,educatic_CTE,becarios,becarios,60837979
1,educatic_CTE,Becarios_formatic,Becarios_h@bitat,73844540
2,educatic_CTE,Chat_de_ayuda,Todos_los_datos_del_sitio_web,83738737
3,educatic_CTE,Consulta_ENP_Diplo_emi_1_y_2,All_Web_Site_Data,66333021
4,educatic_CTE,Dip_Aplicaciones_TIC_a_la_FCPyS,Dip_Aplicaciones_TIC_a_la_FCPyS,53967729
5,educatic_CTE,educatic_unam_mx,educatic,68107963
6,educatic_CTE,educatic_unam_mx,encuentro_educatic,212236976
7,educatic_CTE,educatic_unam_mx,Tu_Aula_Virtual,212237564
8,educatic_CTE,Evaluacion,Evaluacion,140505975
9,educatic_CTE,Evaluacion,Todos_los_datos_del_sitio_web,140509493
